<a href="https://colab.research.google.com/github/amankiitg/DLXR/blob/main/Module_1_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import os
from skimage.transform import resize
from skimage.io import imread
import numpy as np
import matplotlib.pyplot as plt

Categories=list(range(0,10))
print(Categories)
flat_data_arr=[] #input array
target_arr=[] #output array

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


In [ ]:
datadir='/content/drive/My Drive/10000/' 
#path which contains all the categories of images
for i in Categories:
    
    print(f'loading... category : {i}')    
    path=os.path.join(datadir,str(i))    
    for img in os.listdir(path):   
        img_array=imread(os.path.join(path,img))        
        img_resized=resize(img_array,(50,50,3))        
        flat_data_arr.append(img_resized.flatten())        
        target_arr.append(Categories.index(i))    
#         print(f'loaded category:{i} successfully')

flat_data=np.array(flat_data_arr)
target=np.array(target_arr)
df=pd.DataFrame(flat_data) 
#dataframe
df['Target']=target

x=df.iloc[:,:-1] #input data 
y=df.iloc[:,-1] #output data

loading... category : 0
loading... category : 1
loading... category : 2
loading... category : 3
loading... category : 4
loading... category : 5
loading... category : 6
loading... category : 7
loading... category : 8
loading... category : 9


In [ ]:
df.shape

(10000, 7501)

# Training and validation

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.20,random_state=77,stratify=y)
print('Splitted Successfully')

Splitted Successfully


# Score

In [ ]:
import timeit

In [ ]:
from sklearn.model_selection import cross_val_score, cross_validate 
from sklearn import metrics
from sklearn.metrics import make_scorer
from sklearn.preprocessing import LabelBinarizer

def multiclass_roc_auc_score(y_test, y_pred, average="macro"):
  lb = LabelBinarizer()
  lb.fit(y_test)
  y_test = lb.transform(y_test)
  y_pred = lb.transform(y_pred)
  return metrics.roc_auc_score(y_test, y_pred, average=average)

scoring = {'Precision': 'precision_macro',
           'Recall': 'recall_macro',
           'F1-Measure': 'f1_macro',
           'Accuracy': 'accuracy',
           'AUC': make_scorer(multiclass_roc_auc_score)}

def get_cross_validation_output(clf):
  scores = cross_validate(clf, x_train, y_train, scoring=scoring,cv=5)
  df_score = pd.DataFrame(scores).T
  df_score['Mean'] = df_score.mean(axis=1)
  df_score = df_score.T
  cols = list(scoring.keys())
  cols = ['test_'+x for x in cols]
  df_score = pd.DataFrame(df_score.loc['Mean',cols]).T
  df_score.columns = list(scoring.keys())
  return df_score#pd.DataFrame(df_score.loc['Mean',cols]).T

In [ ]:
def get_test_results(model, average='macro'):
    y_pred = model.predict(x_test)
    test_score = {'Precision': metrics.precision_score(y_test, y_pred, average=average),
           'Recall': metrics.recall_score(y_test, y_pred, average=average),
           'F1-Measure': metrics.f1_score(y_test, y_pred, average=average),
           'Accuracy': metrics.accuracy_score(y_test, y_pred),
           'AUC': multiclass_roc_auc_score(y_test, y_pred)}
    return pd.DataFrame.from_dict(test_score,orient='index').T

# Naive Bayes

In [ ]:
from sklearn.naive_bayes import BernoulliNB, MultinomialNB, GaussianNB
clf = MultinomialNB()

In [ ]:
get_cross_validation_output(clf)

,Precision,Recall,F1-Measure,Accuracy,AUC
Mean,0.624722,0.605625,0.608419,0.605625,0.780903


In [ ]:
clf.get_params()

{'alpha': 1.0, 'class_prior': None, 'fit_prior': True}

In [ ]:
clf.fit(x_train, y_train)
get_test_results(clf)

,Precision,Recall,F1-Measure,Accuracy,AUC
0,0.625502,0.6055,0.608034,0.6055,0.780833


# SVM

In [ ]:
from sklearn import svm

tic = timeit.default_timer()
clf = svm.SVC(decision_function_shape='ovr')
display(get_cross_validation_output(clf))
toc = timeit.default_timer()
print('Time',toc - tic)


tic = timeit.default_timer()
clf.fit(x_train, y_train)
display(get_test_results(clf))
toc = timeit.default_timer()
print('Time',toc - tic)

,Precision,Recall,F1-Measure,Accuracy,AUC
Mean,0.823552,0.82175,0.821638,0.82175,0.900972


Time 1313.889453314001


,Precision,Recall,F1-Measure,Accuracy,AUC
0,0.840826,0.839,0.838729,0.839,0.910556


Time 515.3255308919979


In [ ]:
clf.

In [ ]:
tic = timeit.default_timer()
clf = svm.SVC(decision_function_shape='ovo')
display(get_cross_validation_output(clf))
toc = timeit.default_timer()
print('Time',toc - tic)


tic = timeit.default_timer()
clf.fit(x_train, y_train)
display(get_test_results(clf))
toc = timeit.default_timer()
print('Time',toc - tic)

,Precision,Recall,F1-Measure,Accuracy,AUC
Mean,0.823552,0.82175,0.821638,0.82175,0.900972


Time 1430.891205936


,Precision,Recall,F1-Measure,Accuracy,AUC
0,0.840826,0.839,0.838729,0.839,0.910556


Time 383.98781363299986


# Decision Tree Classifier


## Simple DT

In [ ]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=0)

tic = timeit.default_timer()
display(get_cross_validation_output(clf))
toc = timeit.default_timer()
print('Time',toc - tic)


tic = timeit.default_timer()
clf.fit(x_train, y_train)
display(get_test_results(clf))
toc = timeit.default_timer()
print('Time',toc - tic)

,Precision,Recall,F1-Measure,Accuracy,AUC
Mean,0.497762,0.497375,0.497161,0.497375,0.720764


Time 396.0578034380005


,Precision,Recall,F1-Measure,Accuracy,AUC
0,0.499941,0.5015,0.499959,0.5015,0.723056


Time 107.61773737099975


## Ensemble Tree

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import ExtraTreeClassifier
extra_tree = ExtraTreeClassifier(random_state=0)
clf = BaggingClassifier(extra_tree, random_state=0)

tic = timeit.default_timer()
display(get_cross_validation_output(clf))
toc = timeit.default_timer()
print('Time',toc - tic)


tic = timeit.default_timer()
clf.fit(x_train, y_train)
display(get_test_results(clf))
toc = timeit.default_timer()
print('Time',toc - tic)

,Precision,Recall,F1-Measure,Accuracy,AUC
Mean,0.604871,0.60325,0.59758,0.60325,0.779583


Time 67.71583222800018


,Precision,Recall,F1-Measure,Accuracy,AUC
0,0.628961,0.629,0.624133,0.629,0.793889


Time 5.2734677870003


# M​ulti-layer Perceptron

In [ ]:
# One hot encoding output label
from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()
lb.fit(y_train)
y_train_cat = lb.transform(y_train)
y_test_cat = lb.transform(y_test)
y_train_cat

array([[0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1]])

## Basic

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import numpy as np
number_epochs = 100

model = Sequential()
model.add(Dense(750, input_shape=(7500,),activation='sigmoid'))   
model.add(Dense(10, activation='softmax'))
model.summary()


model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['Precision','Recall','accuracy','AUC'])
history=model.fit(x_train, y_train_cat, epochs=number_epochs, validation_split=0.2)

test_score = model.evaluate(x_test, y_test_cat, return_dict=True)
display(pd.DataFrame.from_dict(test_score,orient='index').T)

Model: "sequential_24"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_99 (Dense)            (None, 750)               5625750   
                                                                 
 dense_100 (Dense)           (None, 10)                7510      
                                                                 
Total params: 5,633,260
Trainable params: 5,633,260
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
200/200 [==============================] - 13s 57ms/step - loss: 2.5612 - precision: 0.1094 - recall: 0.0011 - accuracy: 0.1089 - auc: 0.5058 - val_loss: 2.3156 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_accuracy: 0.1081 - val_auc: 0.5315
Epoch 2/100
200/200 [==============================] - 10s 48ms/step - loss: 2.2869 - precision: 0.0000e+00 - recall: 0.0000e+00 - accuracy: 0.1372 - auc: 0.5600 - val_loss

,loss,precision,recall,accuracy,auc
0,1.723187,0.0,0.0,0.2965,0.824985


## Advanced

### 2 Layers

In [ ]:
for activation_fn in ['sigmoid','tanh','relu']:
  model = Sequential()
  model.add(Dense(750, input_shape=(7500,),activation=activation_fn))   
  model.add(Dense(500, activation=activation_fn))
  model.add(Dense(10, activation='softmax'))
  model.summary()


  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['Precision','Recall','accuracy','AUC'])
  model.fit(x_train, y_train_cat, epochs=number_epochs, validation_split=0.2)

  test_score = model.evaluate(x_test, y_test_cat, return_dict=True)
  display(pd.DataFrame.from_dict(test_score,orient='index').T)

Model: "sequential_25"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_101 (Dense)           (None, 750)               5625750   
                                                                 
 dense_102 (Dense)           (None, 500)               375500    
                                                                 
 dense_103 (Dense)           (None, 10)                5010      
                                                                 
Total params: 6,006,260
Trainable params: 6,006,260
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
200/200 [==============================] - 12s 56ms/step - loss: 2.3806 - precision: 0.2500 - recall: 0.0012 - accuracy: 0.1014 - auc: 0.5031 - val_loss: 2.2928 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_accuracy: 0.1081 - val_auc: 0.5475
Epoch 2/100
200/200 [===================

,loss,precision,recall,accuracy,auc
0,1.849148,0.0,0.0,0.271,0.792832


Model: "sequential_26"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_104 (Dense)           (None, 750)               5625750   
                                                                 
 dense_105 (Dense)           (None, 500)               375500    
                                                                 
 dense_106 (Dense)           (None, 10)                5010      
                                                                 
Total params: 6,006,260
Trainable params: 6,006,260
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
200/200 [==============================] - 13s 57ms/step - loss: 2.4612 - precision: 0.1200 - recall: 0.0014 - accuracy: 0.0909 - auc: 0.4926 - val_loss: 2.3809 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_accuracy: 0.1081 - val_auc: 0.5034
Epoch 2/100
200/200 [===================

,loss,precision,recall,accuracy,auc
0,2.348365,0.0,0.0,0.1,0.5


Model: "sequential_27"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_107 (Dense)           (None, 750)               5625750   
                                                                 
 dense_108 (Dense)           (None, 500)               375500    
                                                                 
 dense_109 (Dense)           (None, 10)                5010      
                                                                 
Total params: 6,006,260
Trainable params: 6,006,260
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
200/200 [==============================] - 14s 60ms/step - loss: 2.2522 - precision: 0.5526 - recall: 0.0641 - accuracy: 0.3011 - auc: 0.7233 - val_loss: 1.3970 - val_precision: 0.8616 - val_recall: 0.1906 - val_accuracy: 0.5606 - val_auc: 0.8958
Epoch 2/100
200/200 [===========================

,loss,precision,recall,accuracy,auc
0,0.951303,0.842967,0.824,0.8295,0.958281


### 4 Layers

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import numpy as np

for activation_fn in ['sigmoid','tanh','relu']:
  model = Sequential()
  model.add(Dense(750, input_shape=(7500,),activation=activation_fn))   
  model.add(Dense(500, activation=activation_fn))    
  model.add(Dense(250, activation=activation_fn))    
  model.add(Dense(150, activation=activation_fn))    
  model.add(Dense(10, activation='softmax'))
  model.summary()


  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['Precision','Recall','accuracy','AUC'])
  model.fit(x_train, y_train_cat, epochs=number_epochs, validation_split=0.2)

  test_score = model.evaluate(x_test, y_test_cat, return_dict=True)
  display(pd.DataFrame.from_dict(test_score,orient='index').T)

Model: "sequential_28"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_110 (Dense)           (None, 750)               5625750   
                                                                 
 dense_111 (Dense)           (None, 500)               375500    
                                                                 
 dense_112 (Dense)           (None, 250)               125250    
                                                                 
 dense_113 (Dense)           (None, 150)               37650     
                                                                 
 dense_114 (Dense)           (None, 10)                1510      
                                                                 
Total params: 6,165,660
Trainable params: 6,165,660
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
200/200 [======================

,loss,precision,recall,accuracy,auc
0,2.069762,0.0,0.0,0.1745,0.705964


Model: "sequential_29"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_115 (Dense)           (None, 750)               5625750   
                                                                 
 dense_116 (Dense)           (None, 500)               375500    
                                                                 
 dense_117 (Dense)           (None, 250)               125250    
                                                                 
 dense_118 (Dense)           (None, 150)               37650     
                                                                 
 dense_119 (Dense)           (None, 10)                1510      
                                                                 
Total params: 6,165,660
Trainable params: 6,165,660
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
200/200 [======================

,loss,precision,recall,accuracy,auc
0,2.314271,0.0,0.0,0.1,0.5


Model: "sequential_30"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_120 (Dense)           (None, 750)               5625750   
                                                                 
 dense_121 (Dense)           (None, 500)               375500    
                                                                 
 dense_122 (Dense)           (None, 250)               125250    
                                                                 
 dense_123 (Dense)           (None, 150)               37650     
                                                                 
 dense_124 (Dense)           (None, 10)                1510      
                                                                 
Total params: 6,165,660
Trainable params: 6,165,660
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
200/200 [======================

,loss,precision,recall,accuracy,auc
0,0.833789,0.844746,0.808,0.826,0.964571
